# Overview
In this notebook I save my notes on Source Free Domain adaptation. I am using this [survey](https://arxiv.org/pdf/2302.11803.pdf) as my guide in this journey.

Source Free domain adaptation can be seen as an extension of the well known Unsupervised domain adaptation subfield. The latter relies on the availability of source data which might be considered an unrealistic expectation in practical settings for several reasons such as:

1. Extremely large datasets that introduce saving and sharing issues 
2. Privacy concerns.

# Notations


* Domain roughly speaking represents a dataset $D^s$ with a set of labels $L$ and the data is sampled from a distribution $P(X)$. The source domain is mathematically presented as 
$$D^s = \{ \{X^s, P(X^s), d^s\}, L^s \}$$
It is available during pretraining.

* Target Domain is referred to as a dataset with no labels with a distribution different from the source domain:
                $$D^s = \{ \{X^t, P(X^t), d^t\} \}$$

* The typical setting of SFUDA assumes having the same set of possible 

# Data-Based Approaches
This direction itself divides into 2 familities of approaches: 
1. Reconstruction based: building a representation of the source domain 
2. Focusing on the unannotated data: identifying clusters in the target domain 

DATA BASED APPROACHES ARE MORE CHALLENGING. THEY ARE LEFT FOR LATER

# Model-Based Approaches:
The second research direction is modifying part of the model's parameters. The main 3 sub-directions within this approach are:
1. self-training
2. entropy minimization
3. contrasive learning

## Pseudo-Labeling
The most widely studied approach so far. Pseudo-labeling can be broken into 3 different parts:
1. Prototype generation: Selecting the reliable samples
2. Pseudo Labels assignment: assigning a label to each sample
3. Pseudo labeling filtering: filter unreliable samples

# Do We Really Need to Access The Source Data?
* [paper](https://arxiv.org/pdf/2002.08546.pdf)
* This paper is considered among the pioneering works in SFDA. I tried to focus on the through process:
1. Assuming the target feature extractor is similar to that of the source feature extractor. Then, having a source domain classifier would work well enough.
2. Since we do not have the source distribution, aligning the source data with the target data distributions isn't possible. 
3. Nevertheless, the authors tackle the problem from a different perspective: What would the target feature extractor look like if the domain shift is mitigated ?
4. The authors' answer: high confidence and diversity. Thus, the use of Maximum information loss 
5. The ***IM*** loss is meant as a substitute for domain confusion / domain alignment. Discriminativeness is achieved through pseudo-labels. Pseudo-labels are chosen as follows:
    * centroid of each class is used as the expected vector of each class: $c_k^{0}$ the representation of each centroid is similar to that out 
    * the first p-label is the label of the centroid that minimizes the distance between the sample's embedding and the centroid's
    * the mean of the current clusters are considered the new centroids $c_k^1$
    * The new labels are chosen once again based on the distance between the sample and the centroid
    * so basically 2 iterations of K-Means clustering algorithm
6. Use the standard pseudo labels loss


### Prototype Generation ideas
### Domain Adaptation without Data
* [link](https://arxiv.org/pdf/2007.01524.pdf)
* [code]()

* This paper provides references for the reasons why the source data might not be always available. such as the [following](https://www-file.huawei.com/-/media/corporate/pdf/trust-center/ai-security-whitepaper.pdf)
Each paper's analysis will be broken down to the 3 steps mentioned above:
|
* Prototype generation: For each training sample $x_t$, the entropy is calculated: $$H(x_t) = \frac{1}{\log(N_c)} \sum_{c=1}^{N_c} p(x_t|c) \cdot \log(p (x_t|c))$$ The reliable samples are chosen according to the following criteria. The lower the entropy, the more confident the model is about its prediction. The threshold is:
$$ \mu = max \{ min(H_c) |c \in C \} $$ 
and for each class we know have multiple prototypes, the samples whose entropy is less than the threshold.

* Pseudo labels assignment: The idea here is simple, calculate the average similarity between the features of the given sample and the prototypes of each class. The sample will be assigned the class with the largest similarity score.

* Filtering labels: The process still uses class prototypes. For each sample, they consider the prototype of the labeled class and the prototypes of the 2nd most probable class. The most reliable samples are the ones such that $$max~d(x_t, M_{1p}) \leq min~d(x_t, M_{2p})$$ In other words, the samples such as the distance to the most dissimilar prototype in $M_{1p}$ is less that the distance to the most similar prototype in $M_{2p}$

* Training: training a classifier and a feature extractor using a linear combination of the source pseudo labels, and the filtered ones.

Here is the ![pseudocode](../images/pseudo_labeling_training.png)



# Exploiting the intrinsic Neighbordhood Structure for SFDA
* [paper](https://arxiv.org/pdf/2110.04202.pdf)

* This paper puts forth an important point: The domain shift slightly disturbs the cluster structure in the embedding space. In other words, the parts with the highest density in most clusters are preserved. So the method's main goal is to assign good labels (pseudo-labels) for samples lying in the ambigous clusters

* It is important to note that this hypothesis is based on emperical and experimental results without theoretically-proven results. 


## ProxyMix: Proxy-Based Mixup training with label refinery 
* [link](https://arxiv.org/pdf/2205.14566.pdf)
* [code]()

* The authors explain that generative approachs generally introduce additional parameters. The generative networs end up suffering from the mode collapse problem.
* one interesting idea is to consider the weights used just before the output layer as centroids for the given categories.
* The other idea is to define a hyperparameter $N$ as to consider the $N$ closest points from the centroids of each class.

* AMONG THE IMPORTANT ISSUES ASSOCIATED WITH PSEUDO-LABELING IN a SFUDA setting, is the class imbalance in prototypes and the model's bias towards easier classes.
* Their main strategy isn't completely clear. so I will be back to it later. NEvertheless, it is based on so called classical works: 


# Contrasive Learning: Adaptive Adversarial Network for Source-free Domain Adaptation
* [Paper](https://openaccess.thecvf.com/content/ICCV2021/papers/Xia_Adaptive_Adversarial_Network_for_Source-Free_Domain_Adaptation_ICCV_2021_paper.pdf)

* The main idea is splitting the target domain into source similar and source dissimilar: 
* Use an adversial loss to create domain-invariant features. 
* The new features need to be matched with the actual categories.

# Generalized SFDA: Local Structure Clustering:
* [paper](https://arxiv.org/pdf/2108.01614.pdf)
* The main idea of this paper is quite simple: The domain shift induces a shift in the embedding space. Nevertheless, the general structure still shares a lot of similarities with embedding of source domain samples. The idea is to have similar predictions ofr similar points: classification consistency.

* The classifier is trained to minimize:
$$L = -\frac{1}{n} \sum_{i=1}^n \sum_{k=1}^K \log(p(x_i)\cdot s(N_k)) +  \sum_{c=1}^C KL(\bar p || q_c)$$
where $S(N_k)$ represents the prediction of the $k$-th nearest neighbor, $\bar p = \frac{1}{n} \sum p_c(x_i)$ and $q_c = \frac{1}{C}$
* The first term serves to train the model to produce consistent predictions, while the second term avoids the trivial solutions as well as imbalanced predictions as it awards the model for having a prediction distribution similar to the uniform distribution.

* The paper still suggests interesting ideas for clustering that should be considered further.
* The distance between samples is the cosine distance of their embeddings.  